使用tf.train.Saver会保存运行tf程序所需要的全部信息。

* 然而有时候并不需要某些信息。比如在测试或离线预测时，只需要知道如何从神经网络的输入层经过前向传播计算得到输出层即可，而不需要类似于变量初始化、模型保存等辅助节点信息。
* 将变量取值和计算图结构分成不同的文件存储有时候也不方便。

针对上述情况，tf提供了convert_variables_to_constants函数，通过该函数，可以将计算图中的变量及其取值通过常量的方式保存，这样整个tf计算图可以统一存放在一个文件中。

在保存的时候，通过convert_variables_to_constants函数来指定保存的 **节点名称** 而不是张量的名称，“add:0”是张量的名称而"add"表示的是节点的名称。

#### 1. pb文件的保存方法。

In [1]:
import tensorflow as tf
from tensorflow.python.framework import graph_util

v1 = tf.Variable(tf.constant(1.0, shape=[1]), name = "v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name = "v2")
result = v1 + v2

init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    # 导出当前计算图的GraphDef部分，只需要这一部分就可以完成从输入层到输出层的计算过程
    graph_def = tf.get_default_graph().as_graph_def()
    # 将图中的变量及其取值转化为常量，同时将不必要的节点去掉。注意‘add’是节点名称
    output_graph_def = graph_util.convert_variables_to_constants(sess, graph_def, ['add'])
    # 将导出的模型存入文件
    with tf.gfile.GFile("Saved_model/combined_model.pb", "wb") as f:
           f.write(output_graph_def.SerializeToString())

INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.


#### 2. 加载pb文件。
在读取模型文件获取变量的值的时候，我们需要指定的是 **张量的名称** 而不是节点的名称

In [2]:
from tensorflow.python.platform import gfile
with tf.Session() as sess:
    model_filename = "Saved_model/combined_model.pb"
   
    # 读取保存的模型文件，并将文件解析成对应的GraphDef Protocol Buffer
    with gfile.FastGFile(model_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    
    # 将graph_def中保存的图加载到当前图中。注意‘add:0’是张量名称
    result = tf.import_graph_def(graph_def, return_elements=["add:0"])
    print(sess.run(result))

[array([3.], dtype=float32)]
